# Manipulation.py: RRT

In [1]:
import robotic as ry
import manipulation as manip
import robot_execution as robex
from maze_utils import generate_maze, solve_maze_rrt

First we need to find a path through the maze. To do this we create an empty config and add an XYPhi joint to which we will attach a block representing the box and robot gripper.

Once this in done we can define an end joint state at the end of the maze. Finally we just call the RRT solver and get the path we need to follow in order to solve the puzzle.

In [2]:
ry.params_add({"rrt/stepsize":.05})

C = ry.Config()
generate_maze(C, [.0, .3, .69])
ret = solve_maze_rrt(C, True)

if not ret.feasible:
    print("The RRT solver was unable to find a feasible path.")
    exit()

Now that we have a path for the robot to follow we can use komo to move the robot arm through the maze.

In [3]:
# Define the new config
C = ry.Config()
C.addFile(ry.raiPath("scenarios/pandaSingle.g"))

generate_maze(C, [.0, .3, .69])

bpos = C.getFrame("start").getPosition()
C.addFrame("box") \
    .setPosition(bpos) \
    .setShape(ry.ST.ssBox, size=[.03, .03, .03, .001]) \
    .setColor([0., 0., 1.]) \
    .setContact(1) \
    .setMass(1.)

C.view()

0

In [4]:
# Grab the box and follow the RRT path
M = manip.ManipulationModelling(C)
M.setup_inverse_kinematics()
M.grasp_box(1., "l_gripper", "box", "l_palm", "y")
pose = M.solve()

if M.feasible:
    robot = robex.Robot(C)
    robot.execute_path_blocking(C, pose)
    robot.grasp(C)

    M = manip.ManipulationModelling(C)
    M.setup_multi_phase_komo(len(ret.x), accumulated_collisions=False)
    M.follow_path_on_plane(ret.x, "box")
    path = M.solve()
    
    if M.feasible:
        robot.execute_path_blocking(C, path)
    else:
        print("Path is not feasible!")

C.view(True)

  -- feasible:
     { time: 0.002347, evals: 15, done: 1, feasible: 1, sos: 0.0195872, f: 0, ineq: 0.2, eq: 0.0120614 }
-- kin_physx.cpp:addJoint:298(0) ADDING JOINT l_panda_joint7-box of type rigid with rel [0, 0, 0]
  -- infeasible:
     { time: 0.019579, evals: 6, done: 1, feasible: 0, sos: 2.27622e-08, f: 0, ineq: 0, eq: 18.7427 }
Path is not feasible!


13